In [ ]:
# @title <p>Essential Import
import os, shutil, json
from PIL import Image
from zipfile import ZipFile
import matplotlib.pyplot as plt
import numpy as np, pandas as pd, random as rd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# @title <p>Torch Essential Import
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transforms = T.ToTensor()
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms, download=True)
train_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms, download=True)

# Image test
print(f'Class : {train_dataset[0][1]}')
plt.imshow(train_dataset[0][0].permute(1, 2, 0))

100%|██████████| 9912422/9912422 [00:00<00:00, 115055987.64it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 49829573.77it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 33053540.30it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3923898.82it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
class VGGNet(nn.Module):

  def __init__(self, model_type="VGG16", in_ch=3, out_ch=100):
    super().__init__()
    self.in_ch = in_ch
    self.out_ch = 1000
    self.types = {
        "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
        "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
        "VGG16": [ 64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
        "VGG19": [ 64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"],
    }

    self.conv_layers = self.create_conv_layers(self.types[model_type])

    self.linear = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, self.out_ch),
        )

  def create_conv_layers(self, model_type):
    layers = []
    in_ch = self.in_ch

    for part in model_type:
      if type(part) == int:
        out_ch = part
        layers += [nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=(3, 3), stride=1, padding=1),
                    nn.BatchNorm2d(part),
                    nn.ReLU()]
        in_ch = part

      elif part == "M":
        layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=2)]
    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.conv_layers(x)
    x = x.reshape(x.shape[0], -1)
    x = self.linear(x)
    return x


model = VGGNet(in_ch=1, model_type="VGG19")
x = torch.randn(1, 1, 224, 224)
out = model(x)
out.shape

torch.Size([1, 1000])